# Tushare 数字货币

In [4]:
import pandas as pd
import tushare as ts

In [5]:
pro = ts.pro_api('')
df = pro.coinbar(exchange='huobi', symbol='btcusdt', freq='15min', start_date='20180801', end_date='20180802')

Exception: 抱歉，您没有权限访问该接口，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。